Baseball Prediction: 1 - Data Wrangling
In this notebook we will wrangle data downloaded from www.retrosheet.org into a dataframe suitable for model building. Specifically, for each game, we will calculate some team statistics over their past 162 games.

At the end, we save our dataframe to a file. This file will be the starting point for the next notebook, in which we build our first model.

To use this notebook, you must first download the game logs here: https://www.retrosheet.org/gamelogs/index.html Towards the bottom of the page there are links for ZIP files containing multiple seasons. Download the 5 zip files for 1980-1989, 1990-1999, ... , 2020-2022, decompress them, and then move all of the single season files to a single directory. You will then need the path to that directory for the variable fname below.

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',1000)

In [19]:
# Fill in the path to your data here...
fname = '/Users/willzinkann/Library/CloudStorage/OneDrive-Personal/Active/Baseball_Prediction_Model/game_data/'+'gl2022.txt'
df = pd.read_csv(fname, header=None)

In [20]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160
0,20220407,0,Thu,SDN,NL,1,ARI,NL,1,2,4,51,N,NaN,NaN,NaN,PHO01,35508.0,198,001100000,000000004,29,4,1,0,0,2,0,0,0,7,0,5,1,0,2,0,7,5,4,4,2,0,24,8,0,0,1,0,26,3,0,0,1,3,0,0,1,7,0,6,0,0,1,0,6,6,2,2,0,0,27,9,0,0,2,0,vanol901,Larry Vanover,belld901,Dan Bellino,barbs901,Sean Barber,valej901,Junior Valentine,NaN,(none),NaN,(none),melvb001,Bob Melvin,lovut001,Tony Lovullo,mantj002,Joe Mantiply,suarr002,Robert Suarez,NaN,(none),beers001,Seth Beer,darvy001,Yu Darvish,bumgm001,Madison Bumgarner,nolaa002,Austin Nola,2,machm001,Manny Machado,5,cronj001,Jake Cronenworth,4,voitl001,Luke Voit,10,myerw001,Wil Myers,9,hosme001,Eric Hosmer,3,profj001,Jurickson Profar,7,kim-h002,Ha-Seong Kim,6,grist001,Trent Grisham,8,varsd001,Daulton Varsho,8,martk001,Ketel Marte,4,perad001,David Peralta,7,walkc002,Christian Walker,3,smitp002,Pavin Smith,9,kellc002,Carson Kelly,2,beers001,Seth Beer,10,ellid002,Drew Ellis,5,perdg001,Gerardo Perdomo,6,NaN,Y
1,20220407,0,Thu,CIN,NL,1,ATL,NL,1,6,3,54,N,NaN,NaN,NaN,ATL03,40545.0,181,012003000,001000020,35,10,0,0,1,6,0,1,1,1,0,13,0,0,1,0,5,5,2,2,0,0,27,7,1,0,0,0,31,4,0,0,1,2,0,0,0,5,0,13,0,0,0,0,6,4,6,6,0,0,27,7,0,0,1,0,laynj901,Jerry Layne,wendh902,Hunter Wendelstedt,whitc901,Chad Whitson,hamaa901,Adam Hamari,NaN,(none),NaN,(none),belld002,David Bell,snitb801,Brian Snitker,mahlt001,Tyler Mahle,friem001,Max Fried,santt001,Tony Santillan,farmk001,Kyle Farmer,mahlt001,Tyler Mahle,friem001,Max Fried,indij001,Jonathan India,4,aquia001,Aristides Aquino,9,phamt001,Tommy Pham,7,vottj001,Joey Votto,3,stept001,Tyler Stephenson,2,senzn001,Nick Senzel,8,mousm001,Mike Moustakas,10,farmk001,Kyle Farmer,6,drurb001,Brandon Drury,5,rosae001,Eddie Rosario,9,olsom001,Matt Olson,3,rilea001,Austin Riley,5,ozunm001,Marcell Ozuna,7,albio001,Ozzie Albies,4,duvaa001,Adam Duvall,8,darnt001,Travis d'Arnaud,2,dicka001,Alex Dickerson,10,swand001,Dansby Swanson,6,NaN,Y
2,20220407,0,Thu,MIL,NL,1,CHN,NL,1,4,5,51,D,NaN,NaN,NaN,CHI11,35112.0,198,000100210,00003020x,33,10,4,0,0,4,0,2,1,4,0,9,1,0,2,0,9,3,5,5,0,0,24,10,0,0,1,0,29,8,3,0,1,5,0,1,1,4,0,7,0,1,1,0,6,6,4,4,0,0,27,9,0,0,2,0,barrt901,Ted Barrett,barkl901,Lance Barksdale,lentn901,Nic Lentz,cejan901,Nestor Ceja,NaN,(none),NaN,(none),counc001,Craig Counsell,rossd001,David Ross,givem001,Mychal Givens,ashba003,Aaron Ashby,robed002,David Robertson,happi001,Ian Happ,burnc002,Corbin Burnes,hendk001,Kyle Hendricks,wongk001,Kolten Wong,4,adamw002,Willy Adames,6,yelic001,Christian Yelich,7,mccua001,Andrew McCutchen,10,tellr001,Rowdy Tellez,3,renfh001,Hunter Renfroe,9,narvo001,Omar Narvaez,2,cainl001,Lorenzo Cain,8,petej002,Jace Peterson,5,orter001,Rafael Ortega,10,madrn001,Nick Madrigal,4,contw001,Willson Contreras,2,happi001,Ian Happ,7,schwf001,Frank Schwindel,3,suzus001,Seiya Suzuki,9,heywj001,Jason Heyward,8,wisdp001,Patrick Wisdom,5,hoern001,Nico Hoerner,6,NaN,Y
3,20220407,0,Thu,PIT,NL,1,SLN,NL,1,0,9,51,D,NaN,NaN,NaN,STL10,46256.0,188,000000000,13000104x,30,6,0,0,0,0,0,0,0,2,0,9,0,1,2,0,5,6,9,9,0,0,24,7,2,0,1,0,31,8,2,0,3,9,0,2,1,7,0,5,1,0,1,0,8,4,0,0,1,0,27,11,0,0,3,0,reynj901,Jim Reynolds,ticht901,Todd Tichenor,muchm901,Mike Muchlinski,merzd901,Dan Merzel,NaN,(none),NaN,(none),sheld801,Derek Shelton,marmo801,Oliver Marmol,waina001,Adam Wainwright,brubj001,JT Brubaker,NaN,(none),oneit001,Tyler O'Neill,brubj001,JT Brubaker,waina001,Adam Wainwright,voged001,Daniel Vogelbach,10,reynb001,Bryan Reynolds,8,hayek001,Ke'Bryan Hayes,5,tsuty001,Yoshi Tsutsugo,3,newmk001,Kevin Newman,6,tu

In [21]:
colnames = ['date','dblheader_code','day_of_week','team_v','league_v','game_no_v',
           'team_h','league_h','game_no_h', 'runs_v', 'runs_h','outs_total','day_night',
            'completion_info','forfeit_info','protest_info','ballpark_id','attendance','game_minutes',
            'linescore_v','linescore_h',
           'AB_v','H_v','2B_v','3B_v','HR_v','RBI_v','SH_v','SF_v','HBP_v','BB_v','IBB_v','SO_v',
            'SB_v', 'CS_v','GIDP_v','CI_v','LOB_v',
            'P_num_v','ERind_v','ERteam_v','WP_v','balk_v',
            'PO_v','ASST_v','ERR_v','PB_v','DP_v','TP_v',
           'AB_h', 'H_h', '2B_h', '3B_h', 'HR_h', 'RBI_h', 'SH_h', 'SF_h', 'HBP_h', 'BB_h', 'IBB_h','SO_h',
            'SB_h', 'CS_h', 'GIDP_h', 'CI_h', 'LOB_h',
            'P_num_h', 'ERind_h', 'ERteam_h', 'WP_h', 'balk_h',
            'PO_h', 'ASST_h', 'ERR_h', 'PB_h', 'DP_h', 'TP_h',
            'ump_HB_id', 'ump_HB_name','ump_1B_id', 'ump_1B_name','ump_2B_id', 'ump_2B_name',
            'ump_3B_id', 'ump_3B_name','ump_LF_id', 'ump_LF_name','ump_RF_id', 'ump_RF_name',
            'mgr_id_v', 'mgr_name_v', 'mgr_id_h', 'mgr_name_h',
            'pitcher_id_w','pitcher_name_w','pitcher_id_l','pitcher_name_l','pitcher_id_s','pitcher_name_s',
            'GWRBI_id','GWRBI_name','pitcher_start_id_v','pitcher_start_name_v','pitcher_start_id_h','pitcher_start_name_h',
            'batter1_name_v', 'batter1_id_v', 'batter1_pos_v', 'batter2_name_v', 'batter2_id_v', 'batter2_pos_v',
            'batter3_name_v', 'batter3_id_v', 'batter3_pos_v', 'batter4_name_v', 'batter4_id_v', 'batter4_pos_v',
            'batter5_name_v', 'batter5_id_v', 'batter5_pos_v', 'batter6_name_v', 'batter6_id_v', 'batter6_pos_v',
            'batter7_name_v', 'batter7_id_v', 'batter7_pos_v', 'batter8_name_v', 'batter8_id_v', 'batter8_pos_v',
            'batter9_name_v', 'batter9_id_v', 'batter9_pos_v', 'batter1_name_h', 'batter1_id_h', 'batter1_pos_h',
            'batter2_name_h', 'batter2_id_h', 'batter2_pos_h', 'batter3_name_h', 'batter3_id_h', 'batter3_pos_h',
            'batter4_name_h', 'batter4_id_h', 'batter4_pos_h', 'batter5_name_h', 'batter5_id_h', 'batter5_pos_h',
            'batter6_name_h', 'batter6_id_h', 'batter6_pos_h', 'batter7_name_h', 'batter7_id_h', 'batter7_pos_h',
            'batter8_name_h', 'batter8_id_h', 'batter8_pos_h', 'batter9_name_h', 'batter9_id_h', 'batter9_pos_h',           
           'misc_info','acqui_info'
           ]

In [22]:
df.columns = colnames

In [23]:
df.sample(10)

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info
2358,20221001,0,Sat,PIT,NL,158,SLN,NL,158,3,13,51,N,NaN,NaN,NaN,STL10,46365.0,181,020000010,64000003x,34,7,3,0,0,3,0,0,0,1,0,6,0,0,1,0,5,4,12,12,1,0,24,11,1,0,1,0,33,8,3,0,1,11,0,0,2,8,0,5,0,0,1,0,6,3,3,3,0,0,27,8,1,0,1,0,rehaj901,Jeremie Rehak,roser901,Randy Rosenberg,kulpr901,Ron Kulpa,conrc901,Chris Conroy,NaN,(none),NaN,(none),sheld801,Derek Shelton,marmo801,Oliver Marmol,montj004,Jordan Montgomery,ortil003,Luis Ortiz,NaN,(none),pujoa001,Albert Pujols,ortil003,Luis Ortiz,montj004,Jordan Montgomery,hayek001,Ke'Bryan Hayes,5,reynb001,Bryan Reynolds,8,castr006,Rodolfo Castro,4,andum001,Miguel Andujar,7,newmk001,Kevin Newman,6,castd004,Diego Castillo,3,suwij001,Jack Suwinski,9,delaj004,Jason Delay,2,mitcc002,Cal Mitchell,10,donob001,Brendan Donovan,4,nootl001,Lars Nootbaar,8,goldp001,Paul Goldschmidt,3,arenn001,Nolan Arenado,5,pujoa001,Albert Pujols,10,burla001,Alec Burleson,9,dickc002,Corey Dickerson,7,moliy001,Yadier Molina,2,dejop001,Paul DeJong,6,NaN,Y
1438,20220725,0,Mon,WAS,NL,98,LAN,NL,95,4,1,54,N,NaN,NaN,NaN,LOS03,48647.0,194,000040000,001000000,33,8,0,1,1,4,0,0,1,6,1,9,2,0,1,0,9,6,1,1,0,0,27,5,2,0,0,0,36,8,1,0,1,1,0,0,0,1,0,8,0,0,0,0,9,4,4,4,2,0,27,6,0,0,2,0,torrc901,Carlos Torres,conrc901,Chris Conroy,mosce901,Edwin Moscoso,kulpr901,Ron Kulpa,NaN,(none),NaN,(none),martd002,Dave Martinez,robed001,Dave Roberts,macha003,Andres Machado,gonst001,Tony Gonsolin,finnk001,Kyle Finnegan,hernc005,Cesar Hernandez,espip001,Paolo Espino,gonst001,Tony Gonsolin,roblv001,Victor Robles,8,hernc005,Cesar Hernandez,4,sotoj001,Juan Soto,9,bellj005,Josh Bell,3,cruzn002,Nelson Cruz,10,herny002,Yadiel Hernandez,7,garcl006,Luis Garcia,6,ruizk001,Keibert Ruiz,2,adrie001,Ehire Adrianza,5,bettm001,Mookie Betts,9,turnt001,Trea Turner,6,freef001,Freddie Freeman,3,smitw003,Will Smith,2,lambj001,Jake Lamb,10,muncm001,Max Muncy,5,lux-g001,Gavin Lux,4,bellc002,Cody Bellinger,8,thomt002,Trayce Thompson,7,NaN,Y
1507,20220730,0,Sat,CHN,NL,100,SFN,NL,101,4,5,51,N,NaN,NaN,NaN,SFO03,40971.0,186,000000103,00230000x,38,10,1,0,0,4,0,0,1,1,0,9,0,0,0,0,9,3,5,5,0,0,24,10,0,0,0,0,31,7,1,0,2,5,0,0,1,4,0,10,0,0,0,0,7,5,4,4,0,0,27,12,1,0,0,0,tumpj901,John Tumpane,blakr901,Ryan Blakney,ramoc901,Charlie Ramos,johna901,Adrian Johnson,NaN,(none),NaN,(none),rossd001,David Ross,kaplg001,Gabe Kapler,roget002,Tyler Rogers,smyld001,D

In [24]:
## Table with every game since 1990
df = pd.DataFrame()
for year in range(1990,2023):
    fname = '/Users/willzinkann/Library/CloudStorage/OneDrive-Personal/Active/Baseball_Prediction_Model/game_data/gl' +str(year)+'.txt'
    df_temp = pd.read_csv(fname, header=None)
    df_temp.columns = colnames
    df_temp['season'] = year
    df = pd.concat((df, df_temp))

In [25]:
df.shape

(75939, 162)

In [26]:
df.info(max_cols=100)

<class 'pandas.core.frame.DataFrame'>
Index: 75939 entries, 0 to 2429
Columns: 162 entries, date to season
dtypes: float64(1), int64(83), object(78)
memory usage: 94.4+ MB


In [27]:
## Calculate a few useful columns
df['run_diff'] = df['runs_h']-df['runs_v']
df['home_victory'] = (df['run_diff']>0).astype(int)
df['run_total'] = df['runs_h'].copy()+df['runs_v'].copy()
df['date_dblhead'] = (df['date'].astype(str) + df['dblheader_code'].astype(str)).astype(int)

In [28]:
# Basic Exploration
df.home_victory.mean()

0.5377605709846061

Big picture - have outcome, need features
Would like to start simple, base features on team results over past n games
Need to do a bit of "data wrangling"
First, we need to have each team's games in chronological order
Then can use pandas 'rolling' functionality to aggregate over windows in the past
Immediate goal, get batting average, on-base percentage, and slugging percentage
Need "running values" based on the past, not including the current game

Single games

In [29]:
df_red_sox = df.loc[((df.team_v=='BOS') | (df.team_h=='BOS')), :]

In [30]:
df_red_sox.shape

(5178, 166)

In [31]:
df_red_sox.head(100)

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead
0,19900409,0,Mon,DET,AL,1,BOS,AL,1,2,5,51,D,NaN,NaN,NaN,BOS07,35199.0,167,000002000,31000010x,33,5,1,0,0,2,0,1,0,4,0,6,0,0,0,0,9,3,2,2,0,0,24,8,2,0,0,0,33,8,4,0,0,4,0,0,0,4,2,3,0,0,0,0,8,3,2,2,0,0,27,11,2,0,0,0,barnl901,Larry Barnett,koscg901,Greg Kosc,roe-r901,Rocky Roe,morrd901,Dan Morrison,NaN,(none),NaN,(none),andes101,Sparky Anderson,morgj101,Joe Morgan,clemr001,Roger Clemens,morrj001,Jack Morris,smitl001,Lee Smith,evand002,Dwight Evans,morrj001,Jack Morris,clemr001,Roger Clemens,philt001,Tony Phillips,5,trama001,Alan Trammell,6,whitl001,Lou Whitaker,4,fielc001,Cecil Fielder,3,mosel001,Lloyd Moseby,8,nokem001,Matt Nokes,2,wardg001,Gary Ward,7,lemoc001,Chet Lemon,9,bergd001,Dave Bergman,10,boggw001,Wade Boggs,5,barrm001,Marty Barrett,4,greem001,Mike Greenwell,7,burke001,Ellis Burks,8,evand002,Dwight Evans,10,robib001,Billy Jo Robidoux,3,penat001,Tony Pena,2,reedj003,Jody Reed,6,romik001,Kevin Romine,9,NaN,Y,1990,3,1,7,199004090
10,19900410,0,Tue,DET,AL,2,BOS,AL,2,2,4,51,D,NaN,NaN,NaN,BOS07,15352.0,174,000020000,00400000x,34,7,1,0,0,2,0,0,1,1,0,7,0,0,0,0,7,3,4,4,2,0,24,13,0,0,2,0,31,10,2,0,0,4,1,0,0,8,4,3,1,0,2,0,12,4,2,2,0,0,27,6,0,0,0,0,koscg901,Greg Kosc,roe-r901,Rocky Roe,morrd901,Dan Morrison,barnl901,Larry Barnett,NaN,(none),NaN,(none),andes101,Sparky Anderson,morgj101,Joe Morgan,boddm001,Mike Boddicker,ritzk001,Kevin Ritz,smitl001,Lee Smith,penat001,Tony Pena,ritzk001,Kevin Ritz,boddm001,Mike Boddicker,philt001,Tony Phillips,5,trama001,Alan Trammell,6,whitl001,Lou Whitaker,4,fielc001,Cecil Fielder,3,mosel001,Lloyd Moseby,8,nokem001,Matt Nokes,2,sheel001,Larry Sheets,7,lemoc001,Chet Lemon,9,bergd001,Dave Bergman,10,boggw001,Wade Boggs,5,barrm001,Marty Barrett,4,greem001,Mike Greenwell,7,burke001,Ellis Burks,8,evand002,Dwight Evans,10,robib001,Billy Jo Robidoux,3,penat001,Tony Pena,2,reedj003,Jody Reed,6,romik001,Kevin Romine,9,NaN,Y,1990,2,1,6,199004100
19,19900411,0,Wed,DET,AL,3,BOS,AL,3,2,3,59,D,NaN,NaN,NaN,BOS07,15369.0,204,0000011000,0000101001,36,7,1,0,0,2,1,0,0,1,0,7,0,0,1,0,6,5,3,3,0,0,29,15,0,0,1,0,36,12,3,0,1,3,1,0,1,3,0,6,0,3,1,0,9,4,1,1,1,0,30,14,1,0,1,0,roe-r901,Rocky Roe,morrd901,Dan Morrison,barnl901,Larry Barnett,koscg901,Greg Kosc,NaN,(none),NaN,(none),andes101,Sparky Anderson,morgj101,Joe Morgan,harrg001,Gre

In [32]:
# Write a function to create a team-specific data frame, given the team
def strip_suffix(x, suff):
    if x.endswith(suff):
        return(x[:-len(suff)])
    else:
        return(x)

visit_cols = [col for col in df.columns if not col.endswith('_h')]
visit_cols_stripped = [strip_suffix(col,'_v') for col in visit_cols]
home_cols = [col for col in df.columns if not col.endswith('_v')]
home_cols_stripped = [strip_suffix(col,'_h') for col in home_cols]

## This subsets the game level df by team, to aggregate team statistics easily
## We also create rolling sums with an offset, so that the rollsum number represents
## statistics up to, but not including, the game in question

def create_team_df(team):
    df_team_v = df[(df.team_v==team)]
    opponent = df_team_v['team_h']
    df_team_v = df_team_v[visit_cols]
    df_team_v.columns = visit_cols_stripped
    df_team_v['home_game'] = 0
    df_team_v['opponent'] = opponent

    df_team_h = df[(df.team_h==team)]
    opponent = df_team_h['team_v']
    df_team_h = df_team_h[home_cols]
    df_team_h.columns = home_cols_stripped
    df_team_h['home_game'] = 1
    df_team_h['opponent'] = opponent


    df_team = pd.concat((df_team_h, df_team_v))
    df_team.sort_values(['date', 'game_no'],inplace=True)
    
    for winsize in [162,30]:
        suff = str(winsize)
        for raw_col in ['AB','H','2B','3B','HR','BB','runs','SB','CS','ERR']:
            new_col = 'rollsum_'+raw_col+'_'+suff
            df_team[new_col] = df_team[raw_col].rolling(winsize, closed='left').sum()

        df_team['rollsum_BATAVG_'+suff] = df_team['rollsum_H_'+suff] / df_team['rollsum_AB_'+suff]
        df_team['rollsum_OBP_'+suff] = (df_team['rollsum_H_'+suff] + df_team['rollsum_BB_'+suff]) / (
                                    df_team['rollsum_AB_'+suff]+df_team['rollsum_BB_'+suff])
        df_team['rollsum_SLG_'+suff] = (df_team['rollsum_H_'+suff] + df_team['rollsum_2B_'+suff] 
                                 + 2*df_team['rollsum_3B_'+suff]+ 
                                3*df_team['rollsum_HR_'+suff] ) / (df_team['rollsum_AB_'+suff])
        df_team['rollsum_OBS_'+suff] = df_team['rollsum_OBP_'+suff] + df_team['rollsum_SLG_'+suff]
    
    df_team['season_game'] = df_team['season']*1000 + df_team['game_no']
    df_team.set_index('season_game', inplace=True)
    return(df_team)

In [33]:
df_red_sox = create_team_df('BOS')

In [34]:
df_red_sox.sample(10)

,date,dblheader_code,day_of_week,team,league,game_no,runs,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore,AB,H,2B,3B,HR,RBI,SH,SF,HBP,BB,IBB,SO,SB,CS,GIDP,CI,LOB,P_num,ERind,ERteam,WP,balk,PO,ASST,ERR,PB,DP,TP,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id,mgr_name,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id,pitcher_start_name,batter1_name,batter1_id,batter1_pos,batter2_name,batter2_id,batter2_pos,batter3_name,batter3_id,batter3_pos,batter4_name,batter4_id,batter4_pos,batter5_name,batter5_id,batter5_pos,batter6_name,batter6_id,batter6_pos,batter7_name,batter7_id,batter7_pos,batter8_name,batter8_id,batter8_pos,batter9_name,batter9_id,batter9_pos,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead,home_game,opponent,rollsum_AB_162,rollsum_H_162,rollsum_2B_162,rollsum_3B_162,rollsum_HR_162,rollsum_BB_162,rollsum_runs_162,rollsum_SB_162,rollsum_CS_162,rollsum_ERR_162,rollsum_BATAVG_162,rollsum_OBP_162,rollsum_SLG_162,rollsum_OBS_162,rollsum_AB_30,rollsum_H_30,rollsum_2B_30,rollsum_3B_30,rollsum_HR_30,rollsum_BB_30,rollsum_runs_30,rollsum_SB_30,rollsum_CS_30,rollsum_ERR_30,rollsum_BATAVG_30,rollsum_OBP_30,rollsum_SLG_30,rollsum_OBS_30
season_game,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016054,20160602,0,Thu,BOS,AL,54,7,51,N,NaN,NaN,NaN,BAL12,21534.0,177,000005002,38,12,1,0,1,7,0,0,0,2,0,6,0,1,0,0,6,4,12,12,0,0,24,9,0,0,0,0,nelsj901,Jeff Nelson,diazl901,Laz Diaz,blasc901,Cory Blaser,eddid901,Doug Eddings,NaN,(none),NaN,(none),farrj001,John Farrell,givem001,Mychal Givens,rossr002,Robbie Ross,NaN,(none),machm001,Manny Machado,porcr001,Rick Porcello,bettm001,Mookie Betts,9,pedrd001,Dustin Pedroia,4,bogax001,Xander Bogaerts,6,ortid001,David Ortiz,10,ramih003,Hanley Ramirez,3,shawt001,Travis Shaw,5,swihb001,Blake Swihart,7,younc004,Chris Young,8,vazqc001,Christian Vazquez,2,NaN,Y,2016,5,1,19,201606020,0,BAL,5735.0,1618.0,363.0,43.0,181.0,482.0,863.0,82.0,25.0,92.0,0.282127,0.337783,0.455100,0.792884,1090.0,338.0,72.0,10.0,52.0,109.0,199.0,15.0,2.0,17.0,0.310092,0.372811,0.537615,0.910425
1992150,19920921,0,Mon,BOS,AL,150,5,58,N,NaN,NaN,NaN,DET04,9558.0,202,0010010030,37,9,3,0,2,4,1,0,0,5,1,5,0,0,0,0,8,4,6,6,0,0,28,9,0,0,0,0,cousd901,Derryl Cousins,brinj901,Joe Brinkman,cedeg901,Gary Cederstrom,reilm901,Mike Reilly,NaN,(none),NaN,(none),hobsb101,Butch Hobson,leitm001,Mark Leiter,irvid001,Daryl Irvine,NaN,(none),gladd001,Dan Gladden,gardm002,Mike Gardiner,hatcb001,Billy Hatcher,7,zupcb001,Bob Zupcic,8,coops001,Scott Cooper,5,wedge001,Eric Wedge,10,planp001,Phil Plantier,9,reedj003,Jody Reed,4,lyons001,Steve Lyons,3,valej002,John Valentin,6,flahj001,John Flaherty,2,NaN,Y,1992,1,1,11,199209210,0,DET,5459.0,1349.0,258.0,21.0,82.0,585.0,599.0,44.0,48.0,140.0,0.247115,0.319987,0.347133,0.667120,1038.0,261.0,46.0,3.0,15.0,98.0,104.0,4.0,10.0,33.0,0.251445,0.316021,0.344894,0.660915
2010058,20100606,0,Sun,BOS,AL,58,3,64,D,NaN,NaN,NaN,BAL12,27774.0,260,02000000100,36,8,0,0,1,3,2,1,2,6,1,9,1,0,2,0,11,5,4,4,0,0,31,13,2,0,1,0,hoyej901,James Hoye,bellw901,Wally Bell,diazl901,Laz Diaz,carav901,Vic Carapazza,NaN,(none),NaN,(none),frant001,Terry Francona,hernd002,David Hernandez,okajh001,Hideki Okajima,NaN,(none),markn001,Nick Markakis,lackj001,John Lackey,scutm001,Marco Scutaro,6,pedrd001,Dustin Pedroia,4,ortid001,David Ortiz,10,youkk001,Kevin Youkilis,3,martv001,Victor Martinez,2,belta001,Adrian Beltre,5,hallb001,Bill Hall,7,camem001,Mike Cameron,8,mcdod002,Darnell McDonald,9,NaN,Y,2010,1,1,7,201006060,0,BAL,5583.0,1514.0,338.0,24.0,224.0,651.0,883.0,99.0,27.0,76.0,0.271180,0.347289,0.460684,0.807973,1047.0,291.0,69.0,6.0,41.0,125.0,175.0,6.0,1.0,10.0,0.277937,0.354949,0.472779,0.827728
1990010,19900420,0,Fri,BOS,A

In [35]:
# Make a dictionary that maps a team name to it's data frame

# Create the team level dataframe for each team - put in dict for easy access
team_data_dict = {}
for team in df.team_v.unique():
    team_data_dict[team] = create_team_df(team)

In [36]:
# Go through the rows of the main dataframe, and augment it with home and visiting teams' features


## Create a variety of summarized statistics for each game
## For each game, we look up the home and visiting team in the team
## data dictionary, and then look up the game, and pull the relevant stats

BATAVG_162_h = np.zeros(df.shape[0])
BATAVG_162_v = np.zeros(df.shape[0])
OBP_162_h = np.zeros(df.shape[0])
OBP_162_v = np.zeros(df.shape[0])
SLG_162_h = np.zeros(df.shape[0])
SLG_162_v = np.zeros(df.shape[0])
OBS_162_h = np.zeros(df.shape[0])
OBS_162_v = np.zeros(df.shape[0])
SB_162_h = np.zeros(df.shape[0])
SB_162_v = np.zeros(df.shape[0])
CS_162_h = np.zeros(df.shape[0])
CS_162_v = np.zeros(df.shape[0])
ERR_162_h = np.zeros(df.shape[0])
ERR_162_v = np.zeros(df.shape[0])
BATAVG_30_h = np.zeros(df.shape[0])
BATAVG_30_v = np.zeros(df.shape[0])
OBP_30_h = np.zeros(df.shape[0])
OBP_30_v = np.zeros(df.shape[0])
SLG_30_h = np.zeros(df.shape[0])
SLG_30_v = np.zeros(df.shape[0])
OBS_30_h = np.zeros(df.shape[0])
OBS_30_v = np.zeros(df.shape[0])
SB_30_h = np.zeros(df.shape[0])
SB_30_v = np.zeros(df.shape[0])
CS_30_h = np.zeros(df.shape[0])
CS_30_v = np.zeros(df.shape[0])
ERR_30_h = np.zeros(df.shape[0])
ERR_30_v = np.zeros(df.shape[0])
i=0
for index, row in df.iterrows():
    if i%1000==0:
        print(i)
    home_team = row['team_h']
    visit_team = row['team_v']
    game_index_v = row['season']*1000 + row['game_no_v']
    game_index_h = row['season']*1000 + row['game_no_h']
    BATAVG_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_BATAVG_162']
    BATAVG_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_BATAVG_162']
    OBP_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_OBP_162']
    OBP_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_OBP_162']
    SLG_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SLG_162']
    SLG_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SLG_162']
    OBS_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_OBS_162']
    OBS_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_OBS_162']
    SB_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SB_162']
    SB_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SB_162']
    CS_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_CS_162']
    CS_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_CS_162']
    ERR_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_ERR_162']
    ERR_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_ERR_162']
    BATAVG_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_BATAVG_30']
    BATAVG_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_BATAVG_30']
    OBP_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_OBP_30']
    OBP_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_OBP_30']
    SLG_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SLG_30']
    SLG_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SLG_30']
    OBS_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_OBS_30']
    OBS_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_OBS_30']
    SB_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SB_30']
    SB_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SB_30']
    CS_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_CS_30']
    CS_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_CS_30']
    ERR_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_ERR_30']
    ERR_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_ERR_30']
    i+=1

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000


In [37]:
## We then put the constructed arrays into the main game level dataframe
df['BATAVG_162_h'] = BATAVG_162_h
df['BATAVG_162_v'] = BATAVG_162_v
df['OBP_162_h'] = OBP_162_h
df['OBP_162_v'] = OBP_162_v
df['SLG_162_h'] = SLG_162_h
df['SLG_162_v'] = SLG_162_v
df['OBS_162_h'] = OBS_162_h
df['OBS_162_v'] = OBS_162_v
df['SB_162_h'] = SB_162_h
df['SB_162_v'] = SB_162_v
df['CS_162_h'] = CS_162_h
df['CS_162_v'] = CS_162_v
df['ERR_162_h'] = ERR_162_h
df['ERR_162_v'] = ERR_162_v
df['BATAVG_30_h'] = BATAVG_30_h
df['BATAVG_30_v'] = BATAVG_30_v
df['OBP_30_h'] = OBP_30_h
df['OBP_30_v'] = OBP_30_v
df['SLG_30_h'] = SLG_30_h
df['SLG_30_v'] = SLG_30_v
df['OBS_30_h'] = OBS_30_h
df['OBS_30_v'] = OBS_30_v
df['SB_30_h'] = SB_30_h
df['SB_30_v'] = SB_30_v
df['CS_30_h'] = CS_30_h
df['CS_30_v'] = CS_30_v
df['ERR_30_h'] = ERR_30_h
df['ERR_30_v'] = ERR_30_v

In [38]:
df.sample(5)

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead,BATAVG_162_h,BATAVG_162_v,OBP_162_h,OBP_162_v,SLG_162_h,SLG_162_v,OBS_162_h,OBS_162_v,SB_162_h,SB_162_v,CS_162_h,CS_162_v,ERR_162_h,ERR_162_v,BATAVG_30_h,BATAVG_30_v,OBP_30_h,OBP_30_v,SLG_30_h,SLG_30_v,OBS_30_h,OBS_30_v,SB_30_h,SB_30_v,CS_30_h,CS_30_v,ERR_30_h,ERR_30_v
542,20140510,0,Sat,SLN,NL,37,PIT,NL,36,3,4,51,N,NaN,NaN,NaN,PIT08,34914.0,210,30000000,00040000x,33,7,1,0,0,1,0,0,2,5,0,10,0,0,1,0,10,3,4,4,0,0,24,11,0,0,1,0,34,11,1,0,0,4,0,0,1,2,0,7,1,0,1,0,9,6,3,3,2,0,27,14,0,0,1,0,rackd901,David Rackley,welkb901,Bill Welke,gormb901,Brian Gorman,wolfj901,Jim Wolf,NaN,(none),NaN,(none),mathm001,Mike Matheny,hurdc001,Clint Hurdle,hughj001,Jared Hughes,lynnl001,Lance Lynn,melam001,Mark Melancon,alvap001,Pedro Alvarez,lynnl001,Lance Lynn,volqe001,Edinson Volquez,carpm002,Matt Carpenter,5,peraj001,Jhonny Peralta,6,hollm001,Matt Holliday,7,craia001,Allen Craig,9,adamm002,Matt Adams,3,ellim001,Mark Ellis,4,bourp001,Peter Bourjos,8,cruzt002,Tony Cruz,2,lynnl001,Lance Lynn,1,tabaj002,Jose Tabata,7,harrj002,Josh Harrison,9,walkn001,Neil Walker,4,alvap001,Pedro Alvarez,5,marts002,Starling Marte,8,davii001,Ike Davis,3,sanct001,Tony Sanchez,2,mercj002,Jordy Mercer,6,volqe001,Edinson Volquez,1,NaN,Y,2014,1,1,7,201405100,0.245049,0.265877,0.306713,0.324752,0.391250,0.397919,0.697963,0.722671,95.0,44.0,40.0,22.0,109.0,83.0,0.244918,0.260116,0.307282,0.317333,0.382381,0.382466,0.689664,0.699800,19.0,10.0,5.0,3.0,21.0,16.0
659,19950617,0,Sat,SFN,NL,49,SLN,NL,49,4,1,54,N,NaN,NaN,NaN,STL09,38664.0,132,010201000,100000000,35,10,2,0,1,3,0,0,0,0,0,9,0,1,1,0,4,2,1,1,1,1,27,9,0,0,0,0,31,4,1,0,0,0,0,0,1,3,0,1,1,0,0,0,7,3,4,4,1,0,27,10,0,0,1,0,davig901,Gerry Davis,grege901,Eric Gregg,tatat901,Terry Tata,wintm901,Mike Winters,NaN,(none),NaN,(none),baked002,Dusty Baker,jorgm001,Mike Jorgensen,bautj001,Jose Bautista,urbat001,Tom Urbani,beckr001,Rod Beck,carrm001,Mark Carreon,bautj001,Jose Bautista,urbat001,Tom Urbani,lewid001,Darren Lewis,8,benjm001,Mike Benjamin,6,bondb001,Barry Bonds,7,hillg001,Glenallen Hill,9,carrm001,Mark Carreon,3,thomr003,Robby Thompson,4,scars001,Steve Scarsone,5,manwk001,Kirt Manwaring,2,bautj001,Jose Bautista,1,gilkb001,Bernard Gilkey,7,cromt001,Tripp Cromer,6,jordb001,Brian Jordan,9,lankr001,Ray Lankford,8,saboc001,Chris Sabo,3,coops0

In [39]:
df.to_csv('df_bp1.csv', index=False)

NameError: name 'numpy' is not defined